# Parallel LU-Decomposition using OpenMP

In [17]:
parallel_l_u_decomposition="""

// including the required libraries

#include <iostream>
#include <iomanip>
#include <cmath>
#include <omp.h>
#include <time.h>
#include <stdio.h>
#include <stdlib.h>  
using namespace std;

// -----------------------------------------------------------------------------
// function to print the matrix
void print_matrix(float** matrix, int size)
{
	// for each row
	for (int i = 0; i < size; i++)
	{
		// for each column
		for (int j = 0; j < size; j++)
		{
			cout <<left<<setw(9)<<setprecision(3)<<matrix[i][j]<<left<<setw(9);
		}
		cout << endl;
	}
}
// -----------------------------------------------------------------------------

// -----------------------------------------------------------------------------
// function for LU-Decomposition of matrix A into L and U
void l_u_d(float** a, float** l, float** u, int size)
{
	//initializing a simple lock for parallel region
	omp_lock_t lock;
	omp_init_lock(&lock);

	//for each column
	// parallelizing for loop of LU-Decomposition
	#pragma omp parallel shared(a,l,u)
	{	
		for (int j = 0; j < size; j++)
		{
// -----------------------------------------------------------------------------		
			// finding lower triangular matrix L

			// for each row
			// rows are split into seperate threads for processing
			#pragma omp for schedule(static)
			for (int i = 0; i < size; i++)
			{
				//if i is smaller than j, set l[i][j] to 0
				if (i < j)
				{
					l[i][j] = 0;
					continue;
				}
				
				l[i][j] = a[i][j];
				for (int k = 0; k < j; k++)
				{
					l[i][j] = l[i][j] - l[i][k] * u[k][j];
				}

			}
// -----------------------------------------------------------------------------
			// finding upper triangular matrix U

			// for each row
			// rows are split into seperate threads for processing
			#pragma omp for schedule(static)
			for (int i = 0; i < size; i++)
			{
				//if i is smaller than j, set u's current index to 0
				if (i < j)
				{
					u[j][i] = 0;
					continue;
				}
		
				if (j == i)
				{
					u[j][i] = 1;
					continue;
				}
				
				u[j][i] = a[j][i] / l[j][j];
				for (int k = 0; k < j; k++)
				{
					u[j][i] = u[j][i] - ((l[j][k] * u[k][i]) / l[j][j]);
				}

			}
// -----------------------------------------------------------------------------
		}
	}    // end of parallel region
}
// -----------------------------------------------------------------------------

// -----------------------------------------------------------------------------
//function to initialize the matrices
void initialize_matrices(float** a, float** l, float** u, int size)
{
	//for each row in the 2D array, initialize the values
	//values are processed by seperate threads
	#pragma omp for schedule(static)
	for (int i = 0; i < size; ++i)
	{
		a[i] = new float[size];
		l[i] = new float[size];
		u[i] = new float[size];
	}
}
// -----------------------------------------------------------------------------

// -----------------------------------------------------------------------------
//function to fill the matrix A with random values
void random_fill(float** matrix, int size)
{
	cout<<"Randomly initializing input matrix A ....."<<endl;
	cout<<endl;
  for(int i=0;i<size;i++)
  {
      for(int j=0;j<size;j++)
      {
          matrix[i][j] = (rand()%10)+1;
      }
  }

  // Making the matrix diagonal dominant, so that it is invertible
	int diagonal = 0;
  float sum = 0;
	for (int i = 0; i < size; i++)
	{
		for (int j = 0; j < size; j++)
		{
			sum += abs(matrix[i][j]);   // sum of all values in column j
		}
		
		sum -= abs(matrix[i][diagonal]);  //Removing the diagonal value from the sum

    // For diagonal dominance, diagonal > sum of all other elements
		matrix[i][diagonal] = sum + (rand()%5)+1;

		diagonal++;
    sum = 0;
	}
}
// -----------------------------------------------------------------------------

int main(int argc, char** argv)
{
	cout<<"----- PARALLEL IMPLEMENTATION OF LU DECOMPOSITION USING OPENMP-----"<<endl;
	cout<<endl;

	double runtime;

	int numThreads;    // to set number of  threads
	omp_set_num_threads(10);
	
	//size of matrix
	int size = 100;

	// initalizing the A,L,U matrices
	float** a = new float* [size];
	float** l = new float* [size];
	float** u = new float* [size];
	initialize_matrices(a, l, u, size);

	// filling matrix A with random values
	random_fill(a, size);

	cout << "A Matrix : " << endl;
	print_matrix(a, size);

	// LU decomposition
	runtime = omp_get_wtime();
	l_u_d(a, l, u, size);
	runtime = omp_get_wtime() - runtime;

	//printing L and U matrices
	cout<<endl;
	cout << "Lower triangular matrix (L Matrix) : " << endl;
	print_matrix(l, size);
	cout<<endl;
	cout << "Upper triangular matrix (U Matrix) :" << endl;
	print_matrix(u, size);
	cout<<endl;

	cout << "Runtime : " << runtime <<" s"<< endl;
	return 0;
}
"""

In [18]:
text_file = open("parallel_l_u_decomposition.cpp", "w")
text_file.write(parallel_l_u_decomposition)
text_file.close()

In [19]:
!g++ -o parallel_l_u_decomposition -fopenmp parallel_l_u_decomposition.cpp

In [20]:
!./parallel_l_u_decomposition

----- PARALLEL IMPLEMENTATION OF LU DECOMPOSITION USING OPENMP-----

Randomly initializing input matrix A .....

A Matrix : 
563      7        8        6        4        6        7        3        10       2        3        8        1        10       4        7        1        7        3        7        2        9        8        10       3        1        3        4        8        6        10       3        3        9        10       8        4        7        2        3        10       4        2        10       5        8        9        5        6        1        4        7        2        1        7        4        3        1        7        2        6        6        5        8        7        6        7        10       4        8        5        6        3        6        5        8        5        5        4        1        8        9        7        9        9        5        4        2        5        10       3        1        7        9        10       3        7        7 